# Investigation: AWS API Call Outside Of Organisation

This notebook assists in investigating this detection.

In [ ]:
from scanner_client import ScannerClient
import pandas as pd
import plotly.express as px

# Initialize Scanner Client
client = ScannerClient(api_url="https://api.kast-apse1.scanner.dev", api_key="YOUR_API_KEY")

In [ ]:
query = """
%ingest.source_type="aws:cloudtrail"
and metadata.vendor_name:"AMAZON"
and metadata.product_name:"AWS CloudTrail"
and recipientAccountId:*
and not recipientAccountId:(
  "<approved_account_id_1>" or
  "<approved_account_id_2>" or
  "<approved_account_id_3>"
)
| stats
    min(timestamp) as firstTime,
    max(timestamp) as lastTime,
    count() as eventCount
  by
    recipientAccountId,
    userIdentity.arn,
    eventSource,
    eventName,
    awsRegion
"""

results = client.query(query, time_range_s=86400)
df = pd.DataFrame(results)
df.head()